# BERT

In [2]:
import torch 
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from textpruner import TransformerPruner
import random

# Load model directly
tokenizer = AutoTokenizer.from_pretrained("hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2")
model = AutoModelForSequenceClassification.from_pretrained("hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2",output_attentions=True)

# load the dataset 
ds = load_dataset("hw2942/financial-news-sentiment")


d:\Python\lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [ ]:
from tqdm import tqdm
def get_new_head_mask_randomly(head_mask_previous,pruned_heads,unpruned_heads):
    head = random.randint(0,len(unpruned_heads)-1)
    pruned_heads.append(unpruned_heads[head])
    head_mask_previous[unpruned_heads[head]//12][unpruned_heads[head]%12] = 0
    unpruned_heads.remove(unpruned_heads[head])
    current_head_mask = head_mask_previous
    return current_head_mask,pruned_heads,unpruned_heads
def get_acc(ds,model,tokenizer,device='cuda'):
    total = 0
    right = 0
    model.to(device)
    for data in ds['train']:
        inputs = tokenizer(data['Title'],return_tensors='pt').to(device)
        outputs = model(**inputs)
        total  += 1
        if torch.max(outputs[0][0].softmax(0),dim=0).indices==data['labels']:
            right += 1
    return right/total
def prune_randomly(ds,model,tokenizer):
    head_mask = torch.tensor([[1]*12]*12)
    accs=[]
    pruned_heads=[]
    unpruned_heads=[i for i in range(144)]
    pruner = TransformerPruner(model)
    model.to('cuda')
    accs.append(get_acc(ds,model,tokenizer))
    itera=144/6
    for i in range(int(itera)):
        for _ in range(6):
            head_mask,pruned_heads,unpruned_heads = get_new_head_mask_randomly(head_mask,pruned_heads,unpruned_heads)
        model.to('cuda')
        pruner.prune(head_mask=head_mask,save_model=False)
        acc_score = get_acc(ds,model,tokenizer)
        accs.append(acc_score)
    return accs

In [8]:
# model.save_pretrained('D:\\research\\research4\models\BERT\model')
# tokenizer.save_pretrained('D:\\research\\research4\models\BERT\\tokenizer')
# ds.save_to_disk('D:\\research\\research4\models\BERT\dataset')

Saving the dataset (0/1 shards):   0%|          | 0/2329 [00:00<?, ? examples/s]

In [134]:
prune_random_times=50
accs_random_list=[]
for i in range(prune_random_times):
    model = AutoModelForSequenceClassification.from_pretrained('D:\\research\\research4\models\BERT\model',output_attentions=True)
    accs_random_=prune_randomly(ds,model,tokenizer)
    print(f'第{i}次裁剪实验:',accs_random_)
    accs_random_list.append(accs_random_)

d:\Python\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

d:\Python\lib\site-packages\torch\nn\init.py:453: UserWarning:


第0次裁剪实验: [0.9682267067410906, 0.9695148132245599, 0.9669386002576214, 0.9677973379132675, 0.9686560755689136, 0.9673679690854444, 0.9665092314297982, 0.9660798626019751, 0.9647917561185058, 0.961356805495921, 0.9553456419063976, 0.8557320738514383, 0.8303993130098755, 0.8183769858308286, 0.7866036925719193, 0.7419493344783169, 0.7346500644053242, 0.7020180334907685, 0.6861313868613139, 0.669386002576213, 0.6530699871189352, 0.6316015457277802, 0.5697724345212538, 0.5418634607127523, 0.5375697724345213]
第1次裁剪实验: [0.9682267067410906, 0.9686560755689136, 0.9682267067410906, 0.9673679690854444, 0.965650493774152, 0.965221124946329, 0.9677973379132675, 0.9643623872906827, 0.961356805495921, 0.9622155431515672, 0.9583512237011593, 0.9493344783168742, 0.9377415199656505, 0.9308716187204809, 0.9257191927866036, 0.8802060970373551, 0.7677114641477029, 0.7612709317303564, 0.5607556891369686, 0.5379991412623444, 0.5375697724345213, 0.5375697724345213, 0.5375697724345213, 0.5375697724345213, 0.537

In [135]:
accs_random_list

[[0.9682267067410906,
  0.9695148132245599,
  0.9669386002576214,
  0.9677973379132675,
  0.9686560755689136,
  0.9673679690854444,
  0.9665092314297982,
  0.9660798626019751,
  0.9647917561185058,
  0.961356805495921,
  0.9553456419063976,
  0.8557320738514383,
  0.8303993130098755,
  0.8183769858308286,
  0.7866036925719193,
  0.7419493344783169,
  0.7346500644053242,
  0.7020180334907685,
  0.6861313868613139,
  0.669386002576213,
  0.6530699871189352,
  0.6316015457277802,
  0.5697724345212538,
  0.5418634607127523,
  0.5375697724345213],
 [0.9682267067410906,
  0.9686560755689136,
  0.9682267067410906,
  0.9673679690854444,
  0.965650493774152,
  0.965221124946329,
  0.9677973379132675,
  0.9643623872906827,
  0.961356805495921,
  0.9622155431515672,
  0.9583512237011593,
  0.9493344783168742,
  0.9377415199656505,
  0.9308716187204809,
  0.9257191927866036,
  0.8802060970373551,
  0.7677114641477029,
  0.7612709317303564,
  0.5607556891369686,
  0.5379991412623444,
  0.5375697724

In [136]:

import numpy as np
import pandas as pd

# Generate the data
heads_pruned = [i * 6 for i in range(25)]  # First column with pruned heads count (0, 6, 12, ..., 294)
accuracy_data = accs_random_list # 50 rows of accuracy values (simulating 25 trials for each pruning level)

df = pd.DataFrame(accuracy_data).T  # Transpose so each sublist becomes a column
df.columns = [f"Trial_{i+1}" for i in range(50)]
df.insert(0, "Heads Pruned", heads_pruned)

# Save to Excel
df.to_excel("BERTrandompruned_heads_data_50x25_2group.xlsx", index=False)


In [131]:
import plotly.graph_objects as go
import numpy as np

# 路径到数据文件
box_data_path = "experiments_results/BERTrandompruned_heads_data_50x25_1group.xlsx"
line_data_path = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"

# 读取箱线图数据 (50x25)
box_df = pd.read_excel(box_data_path)

# 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
line_df = pd.read_csv(line_data_path, header=None)
line_data = line_df.squeeze()  # 转换为 Series

# 确保数据对齐，通过选择正确的行和列
heads_pruned = box_df["Heads Pruned"]
box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 50 x 25 的箱线图数据

# 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)

# 计算用于 x 轴标签的修剪百分比
percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)


# Generate data
np.random.seed(0)
heads_pruned = np.arange(0, 150, 6)
accuracy_no_pruning = 0.968226706741091
accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
accuracy_random = box_data

# Calculate percentage pruned for x-axis labels
percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

# Create figure
fig = go.Figure()

# A* Search line
fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

# No Pruning line
fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                         mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

# Random pruning box plot
for i, h in enumerate(heads_pruned):
    fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                         marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

fig.add_trace(go.Scatter(
    x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
    mode='markers',
    marker=dict(color='red'),
    name='Outliers Pruning'  # 设置图例标签
))

fig.update_layout(
    title="Accuracy During Heads Pruning",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    xaxis=dict(
        showgrid=True,
        tickvals=heads_pruned[::3],
        ticktext=[f"{p}%" for h, p in zip(heads_pruned[::3], percent_pruned[::3])],
        tickmode='array'
    ),
    yaxis=dict(showgrid=True, range=[0.5, 1.0]),  # 调整 y 轴的范围
    template="plotly_white",
    width=900,  # 设置图的宽度
    height=600   # 设置图的高度
)

fig.show()


In [137]:
import plotly.graph_objects as go
import numpy as np

# 路径到数据文件
box_data_path = "experiments_results/BERTrandompruned_heads_data_50x25_2group.xlsx"
line_data_path = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"

# 读取箱线图数据 (50x25)
box_df = pd.read_excel(box_data_path)

# 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
line_df = pd.read_csv(line_data_path, header=None)
line_data = line_df.squeeze()  # 转换为 Series

# 确保数据对齐，通过选择正确的行和列
heads_pruned = box_df["Heads Pruned"]
box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 50 x 25 的箱线图数据

# 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)

# 计算用于 x 轴标签的修剪百分比
percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)


# Generate data
np.random.seed(0)
heads_pruned = np.arange(0, 150, 6)
accuracy_no_pruning = 0.968226706741091
accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
accuracy_random = box_data

# Calculate percentage pruned for x-axis labels
percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

# Create figure
fig = go.Figure()

# A* Search line
fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

# No Pruning line
fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                         mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

# Random pruning box plot
for i, h in enumerate(heads_pruned):
    fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                         marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

fig.add_trace(go.Scatter(
    x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
    mode='markers',
    marker=dict(color='red'),
    name='Outliers Pruning'  # 设置图例标签
))

fig.update_layout(
    title="Accuracy During Heads Pruning",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    xaxis=dict(
        showgrid=True,
        tickvals=heads_pruned[::3],
        ticktext=[f"{p}%" for h, p in zip(heads_pruned[::3], percent_pruned[::3])],
        tickmode='array'
    ),
    yaxis=dict(showgrid=True, range=[0.5, 1.0]),  # 调整 y 轴的范围
    template="plotly_white",
    width=900,  # 设置图的宽度
    height=600   # 设置图的高度
)

fig.show()


In [78]:

import plotly.graph_objects as go
import numpy as np

# 路径到数据文件
box_data_path = "experiments_results/BERTrandompruned_heads_data_50x25_concated.xlsx"
line_data_path = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"

# 读取箱线图数据 (100x25)
box_df = pd.read_excel(box_data_path)

# 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
line_df = pd.read_csv(line_data_path, header=None)
line_data = line_df.squeeze()  # 转换为 Series

# 确保数据对齐，通过选择正确的行和列
heads_pruned = box_df["Heads Pruned"]
box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 100 x 25 的箱线图数据

# 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)

# 计算用于 x 轴标签的修剪百分比
percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)


# Generate data
heads_pruned = np.arange(0, 150, 6)
accuracy_no_pruning = 0.968226706741091
accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
accuracy_random = box_data

# Calculate percentage pruned for x-axis labels
percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

# Create figure
fig = go.Figure()

# A* Search line
fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

# No Pruning line
fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                         mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

# Random pruning box plot
for i, h in enumerate(heads_pruned):
    fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                         marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

fig.add_trace(go.Scatter(
    x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
    mode='markers',
    marker=dict(color='red'),
    name='Outliers Pruning'  # 设置图例标签
))

fig.update_layout(
    title="BERT Random Pruning",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    xaxis=dict(
        showgrid=True,
        tickvals=heads_pruned[::3],
        ticktext=[f"{p}%" for h, p in zip(heads_pruned[::3], percent_pruned[::3])],
        tickmode='array'
    ),
    yaxis=dict(showgrid=True, range=[0.5, 1.0]),  # 调整 y 轴的范围
    template="plotly_white",
    width=900,  # 设置图的宽度
    height=600   # 设置图的高度
)

fig.show()


# ALBERT

In [7]:
from datasets import load_dataset
import torch
from tqdm import tqdm
from textpruner import TransformerPruner
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Alireza1044/albert-base-v2-mnli")
model = AutoModelForSequenceClassification.from_pretrained("Alireza1044/albert-base-v2-mnli")
ds = load_dataset("nyu-mll/glue", "mnli")

In [ ]:
# model.save_pretrained('D:\\research\\research4\models\ALBERT\model')
# tokenizer.save_pretrained('D:\\research\\research4\models\ALBERT\\tokenizer')
# ds.save_to_disk('D:\\research\\research4\models\ALBERT\dataset')

In [10]:
from tqdm import tqdm
def get_new_head_mask_randomly(head_mask_previous,pruned_heads,unpruned_heads):
    head = random.randint(0,len(unpruned_heads)-1)
    pruned_heads.append(unpruned_heads[head])
    head_mask_previous[0][unpruned_heads[head]%12] = 0
    unpruned_heads.remove(unpruned_heads[head])
    current_head_mask = head_mask_previous
    return current_head_mask,pruned_heads,unpruned_heads
def get_acc(ds,model,tokenizer,size=len(ds['validation_matched']),device='cuda'):
    count = 0
    cursor = 0
    model.to(device)
    for data in tqdm(ds['validation_matched']):
        cursor += 1
        inputs = tokenizer(data['premise'],data['hypothesis'],return_tensors='pt').to(device) # 这里会自动在两个输入的语句中插入分隔符
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if predicted_class_id == data['label']:
            count += 1
        if cursor > size:
            break
    return count / size
def prune_randomly(ds,model,tokenizer):
    head_mask = torch.tensor([[1]*12])
    accs=[]
    pruned_heads=[]
    unpruned_heads=[i for i in range(12)]
    pruner = TransformerPruner(model)
    model.to('cuda')
    accs.append(get_acc(ds,model,tokenizer))
    for _ in range(12):
        head_mask,pruned_heads,unpruned_heads = get_new_head_mask_randomly(head_mask,pruned_heads,unpruned_heads)
        model.to('cuda')
        pruner.prune(head_mask=head_mask,save_model=False)
        acc_score = get_acc(ds,model,tokenizer)
        accs.append(acc_score)
    return accs

In [12]:
import random
prune_random_times=25
accs_random_list_ALBERT=[]
for i in range(prune_random_times):
    model = AutoModelForSequenceClassification.from_pretrained('D:\\research\\research4\models\ALBERT\model',output_attentions=True)
    accs_random_=prune_randomly(ds,model,tokenizer)
    print(f'第{i}次裁剪实验:',accs_random_)
    accs_random_list_ALBERT.append(accs_random_)

d:\Python\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cpu")
100%|███

第0次裁剪实验: [0.8448293428425879, 0.8440142638818136, 0.33245033112582784, 0.3321446765155374, 0.321650534895568, 0.33968415690269993, 0.3544574630667346, 0.3402954661232807, 0.31665817626082526, 0.3228731533367295, 0.3495669893020886, 0.35354049923586345, 0.31818644931227713]


100%|██████████| 9815/9815 [00:54<00:00, 180.25it/s]


第1次裁剪实验: [0.8448293428425879, 0.32908813041263374, 0.3142129393785023, 0.3106469689251146, 0.3171676006113092, 0.30932246561385635, 0.3254202750891493, 0.3249108507386653, 0.3205298013245033, 0.33204279164544065, 0.3238920020376974, 0.3183902190524707, 0.31818644931227713]


100%|██████████| 9815/9815 [00:53<00:00, 184.10it/s]


...

第22次裁剪实验: [0.8448293428425879, 0.8274070300560367, 0.8173204279164544, 0.7181864493122772, 0.5777890983188997, 0.5535404992358635, 0.4431991849210392, 0.35048395313295977, 0.3273560876209883, 0.3272542027508915, 0.3191034131431482, 0.317269485481406, 0.31818644931227713]


100%|██████████| 9815/9815 [00:53<00:00, 182.57it/s]


第23次裁剪实验: [0.8448293428425879, 0.8172185430463577, 0.7835965359144167, 0.719714722363729, 0.5560876209882832, 0.32990320937340806, 0.3285787060621498, 0.3544574630667346, 0.35455934793683136, 0.31920529801324504, 0.3184921039225675, 0.31818644931227713, 0.31818644931227713]


100%|██████████| 9815/9815 [00:53<00:00, 182.26it/s]

第24次裁剪实验: [0.8448293428425879, 0.31451859398879267, 0.3272542027508915, 0.32745797249108505, 0.34212939378502294, 0.32868059093224655, 0.3287824758023434, 0.3273560876209883, 0.3336729495669893, 0.32847682119205296, 0.32786551197147223, 0.3337748344370861, 0.31818644931227713]


In [13]:
accs_random_list_ALBERT

[[0.8448293428425879,
  0.8440142638818136,
  0.33245033112582784,
  0.3321446765155374,
  0.321650534895568,
  0.33968415690269993,
  0.3544574630667346,
  0.3402954661232807,
  0.31665817626082526,
  0.3228731533367295,
  0.3495669893020886,
  0.35354049923586345,
  0.31818644931227713],
 [0.8448293428425879,
  0.32908813041263374,
  0.3142129393785023,
  0.3106469689251146,
  0.3171676006113092,
  0.30932246561385635,
  0.3254202750891493,
  0.3249108507386653,
  0.3205298013245033,
  0.33204279164544065,
  0.3238920020376974,
  0.3183902190524707,
  0.31818644931227713],
 [0.8448293428425879,
  0.7758532857870606,
  0.32440142638818137,
  0.310443199184921,
  0.3239938869077942,
  0.3315333672949567,
  0.3236882322975038,
  0.3283749363219562,
  0.32603158430973,
  0.3398879266428935,
  0.31808456444218036,
  0.31818644931227713,
  0.31818644931227713],
 [0.8448293428425879,
  0.8274070300560367,
  0.7738155883851248,
  0.699235863474274,
  0.40295466123280693,
  0.3429444727457972

In [14]:
import numpy as np
import pandas as pd

# Generate the data
heads_pruned = [i * 12 for i in range(13)]  # First column with pruned heads count (0, 6, 12, ..., 294)
accuracy_data = accs_random_list_ALBERT # 50 rows of accuracy values (simulating 25 trials for each pruning level)

df = pd.DataFrame(accuracy_data).T  # Transpose so each sublist becomes a column
df.columns = [f"Trial_{i+1}" for i in range(25)]
df.insert(0, "Heads Pruned", heads_pruned)

# Save to Excel
df.to_excel("ALBERTrandompruned_heads_data_25x13_1group.xlsx", index=False)


In [77]:

import plotly.graph_objects as go
import numpy as np

# 路径到数据文件
box_data_path = "experiments_results/ALBERTrandompruned_heads_data_25x13_1group.xlsx"
line_data_path = "experiments_results/pruned_heads_accuracy(ALBERT)basedonGreedyGnorm.csv"

# 读取箱线图数据 (100x25)
box_df = pd.read_excel(box_data_path)

# 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
line_df = pd.read_csv(line_data_path, header=None)
line_data = line_df.squeeze()  # 转换为 Series

# 确保数据对齐，通过选择正确的行和列
heads_pruned = box_df["Heads Pruned"]
box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 25 x 13 的箱线图数据

# 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
line_data_sampled = line_data.iloc[1:].reset_index(drop=True)

# 计算用于 x 轴标签的修剪百分比
percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)


# Generate data
heads_pruned = np.arange(0, 150, 12)
accuracy_no_pruning = 0.8448293428425879
accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
accuracy_random = box_data

# Calculate percentage pruned for x-axis labels
percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

# Create figure
fig = go.Figure()

# A* Search line
fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

# No Pruning line
fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                         mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

# Random pruning box plot
for i, h in enumerate(heads_pruned):
    fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                         marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

fig.add_trace(go.Scatter(
    x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
    mode='markers',
    marker=dict(color='red'),
    name='Outliers Pruning'  # 设置图例标签
))

fig.update_layout(
    title="ALBERT Random Pruning",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    xaxis=dict(
        showgrid=True,
        tickvals=heads_pruned[::],
        ticktext=[f"{p}%" for h, p in zip(heads_pruned[::], percent_pruned[::])],
        tickmode='array'
    ),
    yaxis=dict(showgrid=True, range=[0.28, 0.9]),  # 调整 y 轴的范围
    template="plotly_white",
    width=900,  # 设置图的宽度
    height=600   # 设置图的高度
)

fig.show()


# ROBERTA

In [24]:
# Load model directly
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")
ds = load_dataset("rahmaabusalma/tweets_sentiment_analysis")

d:\Python\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [25]:
model.save_pretrained('D:\\research\\research4\models\ROBERTA\model')

In [37]:
from tqdm import tqdm
def get_new_head_mask_randomly(head_mask_previous,pruned_heads,unpruned_heads):
    head = random.randint(0,len(unpruned_heads)-1)
    pruned_heads.append(unpruned_heads[head])
    head_mask_previous[unpruned_heads[head]//16][unpruned_heads[head]%16] = 0
    unpruned_heads.remove(unpruned_heads[head])
    current_head_mask = head_mask_previous
    return current_head_mask,pruned_heads,unpruned_heads
def get_acc(ds,model,tokenizer,size=len(ds['validation']),device='cuda'):
    count = 0
    cursor = 0
    model.to(device)
    for data in ds['validation']:
        if data['label']==1:
            continue
        cursor += 1
        inputs = tokenizer(data['text'],return_tensors='pt').to(device) # 这里会自动在两个输入的语句中插入分隔符
        with torch.no_grad():
            predicted_class_id = model(**inputs)
        if torch.max(predicted_class_id[0],dim=1).indices[0] == (1 if data['label']==2 else 0):
            count += 1
        if cursor > size:
            break
    return count / size
def prune_randomly(ds,model,tokenizer):
    head_mask = torch.tensor([[1]*16]*24)
    accs=[]
    pruned_heads=[]
    unpruned_heads=[i for i in range(384)]
    pruner = TransformerPruner(model)
    model.to('cuda')
    accs.append(get_acc(ds,model,tokenizer,1000))
    itera=384/16
    for __ in range(int(itera)):
        for __ in range(16):
            head_mask,pruned_heads,unpruned_heads = get_new_head_mask_randomly(head_mask,pruned_heads,unpruned_heads)
        model.to('cuda')
        pruner.prune(head_mask=head_mask,save_model=False)
        acc_score = get_acc(ds,model,tokenizer,1000)
        accs.append(acc_score)
    return accs

In [ ]:
prune_random_times=25
accs_random_list_ROBERTA=[]
for i in range(prune_random_times):
    model = AutoModelForSequenceClassification.from_pretrained('D:\\research\\research4\models\ROBERTA\model',output_attentions=True)
    accs_random_=prune_randomly(ds,model,tokenizer)
    print(f'第{i}次裁剪实验:',accs_random_)
    accs_random_list_ROBERTA.append(accs_random_)

d:\Python\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

d:\Python\lib\site-packages\torch\nn\init.py:453: UserWarning:


第0次裁剪实验: [0.878, 0.878, 0.884, 0.888, 0.889, 0.889, 0.884, 0.882, 0.873, 0.869, 0.867, 0.851, 0.857, 0.842, 0.858, 0.832, 0.805, 0.698, 0.666, 0.639, 0.571, 0.55, 0.55, 0.55, 0.55]
第1次裁剪实验: [0.878, 0.889, 0.89, 0.881, 0.89, 0.885, 0.889, 0.893, 0.895, 0.887, 0.877, 0.883, 0.881, 0.876, 0.88, 0.885, 0.859, 0.856, 0.828, 0.801, 0.773, 0.691, 0.585, 0.55, 0.55]
第2次裁剪实验: [0.878, 0.888, 0.884, 0.883, 0.882, 0.888, 0.885, 0.893, 0.882, 0.884, 0.892, 0.882, 0.877, 0.874, 0.863, 0.836, 0.819, 0.654, 0.684, 0.597, 0.552, 0.55, 0.594, 0.55, 0.55]
第3次裁剪实验: [0.878, 0.888, 0.89, 0.886, 0.876, 0.883, 0.883, 0.878, 0.877, 0.876, 0.875, 0.876, 0.876, 0.87, 0.832, 0.836, 0.799, 0.758, 0.622, 0.631, 0.606, 0.548, 0.55, 0.55, 0.55]
第4次裁剪实验: [0.878, 0.888, 0.887, 0.893, 0.89, 0.889, 0.89, 0.878, 0.883, 0.879, 0.878, 0.874, 0.868, 0.869, 0.86, 0.859, 0.852, 0.799, 0.772, 0.689, 0.648, 0.577, 0.55, 0.55, 0.55]
第5次裁剪实验: [0.878, 0.885, 0.884, 0.883, 0.878, 0.878, 0.876, 0.872, 0.877, 0.874, 0.878, 0.875, 0.86

In [40]:
accs_random_list_ROBERTA

[[0.878,
  0.878,
  0.884,
  0.888,
  0.889,
  0.889,
  0.884,
  0.882,
  0.873,
  0.869,
  0.867,
  0.851,
  0.857,
  0.842,
  0.858,
  0.832,
  0.805,
  0.698,
  0.666,
  0.639,
  0.571,
  0.55,
  0.55,
  0.55,
  0.55],
 [0.878,
  0.889,
  0.89,
  0.881,
  0.89,
  0.885,
  0.889,
  0.893,
  0.895,
  0.887,
  0.877,
  0.883,
  0.881,
  0.876,
  0.88,
  0.885,
  0.859,
  0.856,
  0.828,
  0.801,
  0.773,
  0.691,
  0.585,
  0.55,
  0.55],
 [0.878,
  0.888,
  0.884,
  0.883,
  0.882,
  0.888,
  0.885,
  0.893,
  0.882,
  0.884,
  0.892,
  0.882,
  0.877,
  0.874,
  0.863,
  0.836,
  0.819,
  0.654,
  0.684,
  0.597,
  0.552,
  0.55,
  0.594,
  0.55,
  0.55],
 [0.878,
  0.888,
  0.89,
  0.886,
  0.876,
  0.883,
  0.883,
  0.878,
  0.877,
  0.876,
  0.875,
  0.876,
  0.876,
  0.87,
  0.832,
  0.836,
  0.799,
  0.758,
  0.622,
  0.631,
  0.606,
  0.548,
  0.55,
  0.55,
  0.55],
 [0.878,
  0.888,
  0.887,
  0.893,
  0.89,
  0.889,
  0.89,
  0.878,
  0.883,
  0.879,
  0.878,
  0.874,
  0.868

In [42]:
import numpy as np
import pandas as pd

# Generate the data
heads_pruned = [i * 16 for i in range(25)]  # First column with pruned heads count (0, 6, 12, ..., 294)
accuracy_data = accs_random_list_ROBERTA # 50 rows of accuracy values (simulating 25 trials for each pruning level)

df = pd.DataFrame(accuracy_data).T  # Transpose so each sublist becomes a column
df.columns = [f"Trial_{i+1}" for i in range(25)]
df.insert(0, "Heads Pruned", heads_pruned)

# Save to Excel
df.to_excel("ROBERTArandompruned_heads_data_25x25_1group.xlsx", index=False)


In [75]:

import plotly.graph_objects as go
import numpy as np

# 路径到数据文件
box_data_path = "experiments_results/ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path = "experiments_results/pruned_heads_accuracy(ROBERTA)basedonGreedyGnorm.csv"

# 读取箱线图数据 (100x25)
box_df = pd.read_excel(box_data_path)

# 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
line_df = pd.read_csv(line_data_path, header=None)
line_data = line_df.squeeze()  # 转换为 Series

# 确保数据对齐，通过选择正确的行和列
heads_pruned = box_df["Heads Pruned"]
box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 25 x 13 的箱线图数据

# 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
line_data_sampled = line_data.iloc[1::16].reset_index(drop=True)

# 计算用于 x 轴标签的修剪百分比
percent_pruned = (heads_pruned / 384 * 100).round(0).astype(int)


# Generate data
heads_pruned = np.arange(0, 384, 16)
accuracy_no_pruning = 0.878
accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
accuracy_random = box_data

# Calculate percentage pruned for x-axis labels
percent_pruned = ((heads_pruned / 384) * 100).round(0).astype(int)

# Create figure
fig = go.Figure()

# A* Search line
fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

# No Pruning line
fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                         mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

# Random pruning box plot
for i, h in enumerate(heads_pruned):
    fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                         marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

fig.add_trace(go.Scatter(
    x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
    mode='markers',
    marker=dict(color='red'),
    name='Outliers Pruning'  # 设置图例标签
))

fig.update_layout(
    title="RoBERTa Random Pruning",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    xaxis=dict(
        showgrid=True,
        tickvals=heads_pruned[::],
        ticktext=[f"{p}%" for h, p in zip(heads_pruned[::], percent_pruned[::])],
        tickmode='array'
    ),
    yaxis=dict(showgrid=True, range=[0.5, 0.9]),  # 调整 y 轴的范围
    template="plotly_white",
    width=900,  # 设置图的宽度
    height=600   # 设置图的高度
)

fig.show()


# XLM-ROBERTA

In [5]:
import torch 
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from textpruner import TransformerPruner
import random

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
model = AutoModelForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")
ds = load_dataset("papluca/language-identification")

In [ ]:
from tqdm import tqdm
def get_new_head_mask_randomly(head_mask_previous,pruned_heads,unpruned_heads):
    head = random.randint(0,len(unpruned_heads)-1)
    pruned_heads.append(unpruned_heads[head])
    head_mask_previous[unpruned_heads[head]//12][unpruned_heads[head]%12] = 0
    unpruned_heads.remove(unpruned_heads[head])
    current_head_mask = head_mask_previous
    return current_head_mask,pruned_heads,unpruned_heads
def get_acc(ds,model,tokenizer,size=len(ds['validation']),device='cuda'):
    count = 0
    id2lang = model.config.id2label
    cursor = 0
    model.to(device)
    for data in ds['validation']:
        cursor += 1
        inputs = tokenizer(data['text'], padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        preds = torch.softmax(logits, dim=-1)
        vals, idx = torch.max(preds, dim=1)
        if id2lang[idx.item()]==data['labels']:
            count += 1
        if cursor > size:
            break
    return count / size
def prune_randomly(ds,model,tokenizer):
    head_mask = torch.tensor([[1]*12]*12)
    accs=[]
    pruned_heads=[]
    unpruned_heads=[i for i in range(144)]
    pruner = TransformerPruner(model)
    model.to('cuda')
    accs.append(get_acc(ds,model,tokenizer))
    itera=144/6
    for i in range(int(itera)):
        for _ in range(6):
            head_mask,pruned_heads,unpruned_heads = get_new_head_mask_randomly(head_mask,pruned_heads,unpruned_heads)
        model.to('cuda')
        pruner.prune(head_mask=head_mask,save_model=False)
        acc_score = get_acc(ds,model,tokenizer,3000)
        accs.append(acc_score)
    return accs

In [2]:
model.save_pretrained('D:\\research\\research4\models\XLM_ROBERTA\model')

In [9]:
prune_random_times=25
accs_random_list_XLM_ROBERTA=[]
for i in range(prune_random_times):
    model = AutoModelForSequenceClassification.from_pretrained('D:\\research\\research4\models\XLM_ROBERTA\model',output_attentions=True)
    accs_random_=prune_randomly(ds,model,tokenizer)
    print(f'第{i}次裁剪实验:',accs_random_)
    accs_random_list_XLM_ROBERTA.append(accs_random_)

d:\Python\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cpu")
d:\Pytho

第0次裁剪实验: [0.9973, 0.9976666666666667, 0.9966666666666667, 0.9966666666666667, 0.9956666666666667, 0.996, 0.993, 0.9943333333333333, 0.967, 0.8336666666666667, 0.8256666666666667, 0.42033333333333334, 0.22066666666666668, 0.08, 0.084, 0.075, 0.067, 0.052, 0.052, 0.052, 0.052, 0.052, 0.052, 0.052, 0.052]
第1次裁剪实验: [0.9973, 0.998, 0.9976666666666667, 0.998, 0.9976666666666667, 0.9976666666666667, 0.9973333333333333, 0.997, 0.9966666666666667, 0.969, 0.9676666666666667, 0.9516666666666667, 0.6183333333333333, 0.27366666666666667, 0.20533333333333334, 0.19166666666666668, 0.13233333333333333, 0.053, 0.052, 0.052, 0.052, 0.052, 0.052, 0.052, 0.052]
第2次裁剪实验: [0.9973, 0.998, 0.998, 0.998, 0.9983333333333333, 0.998, 0.997, 0.9966666666666667, 0.9963333333333333, 0.994, 0.9886666666666667, 0.9686666666666667, 0.795, 0.5363333333333333, 0.10233333333333333, 0.076, 0.07366666666666667, 0.07633333333333334, 0.073, 0.052, 0.052, 0.052, 0.052, 0.052, 0.052]
第3次裁剪实验: [0.9973, 0.9986666666666667, 0.998,

In [10]:

import numpy as np
import pandas as pd

# Generate the data
heads_pruned = [i * 6 for i in range(25)]  # First column with pruned heads count (0, 6, 12, ..., 294)
accuracy_data = accs_random_list_XLM_ROBERTA # 50 rows of accuracy values (simulating 25 trials for each pruning level)

df = pd.DataFrame(accuracy_data).T  # Transpose so each sublist becomes a column
df.columns = [f"Trial_{i+1}" for i in range(25)]
df.insert(0, "Heads Pruned", heads_pruned)

# Save to Excel
df.to_excel("XLM_ROBERTArandompruned_heads_data_25x25_1group.xlsx", index=False)


In [76]:
import plotly.graph_objects as go
import numpy as np

# 路径到数据文件
box_data_path = "experiments_results/XLM_ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path = "experiments_results/pruned_heads_accuracy(XLM_ROBERTA)basedonGreedyGnorm.csv"

# 读取箱线图数据 (50x25)
box_df = pd.read_excel(box_data_path)

# 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
line_df = pd.read_csv(line_data_path, header=None)
line_data = line_df.squeeze()  # 转换为 Series

# 确保数据对齐，通过选择正确的行和列
heads_pruned = box_df["Heads Pruned"]
box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 50 x 25 的箱线图数据

# 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)

# 计算用于 x 轴标签的修剪百分比
percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)


# Generate data
np.random.seed(0)
heads_pruned = np.arange(0, 150, 6)
accuracy_no_pruning = 0.9973
accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
accuracy_random = box_data

# Calculate percentage pruned for x-axis labels
percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

# Create figure
fig = go.Figure()

fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

# No Pruning line
fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                         mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

# Random pruning box plot
for i, h in enumerate(heads_pruned):
    fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                         marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

fig.add_trace(go.Scatter(
    x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
    mode='markers',
    marker=dict(color='red'),
    name='Outliers Pruning'  # 设置图例标签
))

fig.update_layout(
    title="XLM-RoBERTa Random Pruning",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    xaxis=dict(
        showgrid=True,
        tickvals=heads_pruned[::3],
        ticktext=[f"{p}%" for h, p in zip(heads_pruned[::3], percent_pruned[::3])],
        tickmode='array'
    ),
    yaxis=dict(showgrid=True, range=[0.0, 1.0]),  # 调整 y 轴的范围
    template="plotly_white",
    width=900,  # 设置图的宽度
    height=600   # 设置图的高度
)

fig.show()


# Combine

In [19]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots


box_data_path_BERT = "experiments_results/BERTrandompruned_heads_data_50x25_concated.xlsx"
line_data_path_BERT = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"
box_data_path_ALBERT = "experiments_results/ALBERTrandompruned_heads_data_25x13_1group.xlsx"
line_data_path_ALBERT = "experiments_results/pruned_heads_accuracy(ALBERT)basedonGreedyGnorm.csv"
box_data_path_ROBERTA = "experiments_results/ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_ROBERTA = "experiments_results/pruned_heads_accuracy(ROBERTA)basedonGreedyGnorm.csv"
box_data_path_XLM = "experiments_results/XLM_ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_XLM = "experiments_results/pruned_heads_accuracy(XLM_ROBERTA)basedonGreedyGnorm.csv"


import plotly.graph_objects as go
import numpy as np

# 路径到数据文件
box_data_path = "experiments_results/BERTrandompruned_heads_data_50x25_concated.xlsx"
line_data_path = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"

# 读取箱线图数据 (100x25)
box_df = pd.read_excel(box_data_path_BERT)

# 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
line_df = pd.read_csv(line_data_path_BERT, header=None)
line_data = line_df.squeeze()  # 转换为 Series

# 确保数据对齐，通过选择正确的行和列
heads_pruned = box_df["Heads Pruned"]
box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 100 x 25 的箱线图数据

# 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)

# 计算用于 x 轴标签的修剪百分比
percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)


# Generate data
heads_pruned = np.arange(0, 150, 6)
accuracy_no_pruning = 0.968226706741091
accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
accuracy_random = box_data

# Calculate percentage pruned for x-axis labels
percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

# Create figure
fig_BERT = go.Figure()

# A* Search line
fig_BERT.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

# No Pruning line
fig_BERT.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                         mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

# Random pruning box plot
for i, h in enumerate(heads_pruned):
    fig_BERT.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                         marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

fig_BERT.add_trace(go.Scatter(
    x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
    mode='markers',
    marker=dict(color='red'),
    name='Outliers Pruning'  # 设置图例标签
))

fig_BERT.update_layout(
    title="Accuracy During Heads Pruning",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    xaxis=dict(
        showgrid=True,
        tickvals=heads_pruned[::3],
        ticktext=[f"{p}%" for h, p in zip(heads_pruned[::3], percent_pruned[::3])],
        tickmode='array'
    ),
    yaxis=dict(showgrid=True, range=[0.5, 1.0]),  # 调整 y 轴的范围
    template="plotly_white",
    width=900,  # 设置图的宽度
    height=600   # 设置图的高度
)

fig_BERT.show()




In [74]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# 读取数据 (以 BERT 数据为例，其他模型数据类似)
# 数据路径
box_data_path_BERT = "experiments_results/BERTrandompruned_heads_data_50x25_concated.xlsx"
line_data_path_BERT = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"
box_data_path_ALBERT = "experiments_results/ALBERTrandompruned_heads_data_25x13_1group.xlsx"
line_data_path_ALBERT = "experiments_results/pruned_heads_accuracy(ALBERT)basedonGreedyGnorm.csv"
box_data_path_ROBERTA = "experiments_results/ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_ROBERTA = "experiments_results/pruned_heads_accuracy(ROBERTA)basedonGreedyGnorm.csv"
box_data_path_XLM = "experiments_results/XLM_ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_XLM = "experiments_results/pruned_heads_accuracy(XLM_ROBERTA)basedonGreedyGnorm.csv"

# 创建 4 个子图
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("BERT", "ALBERT", "RoBERTa", "XLM-RoBERTa"),
    shared_yaxes=True,
    vertical_spacing=0.1
)

def create_model_plots_BERT(fig, row, col, box_data_path, line_data_path):
    box_df = pd.read_excel(box_data_path)

    # 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
    line_df = pd.read_csv(line_data_path, header=None)
    line_data = line_df.squeeze()  # 转换为 Series

    # 确保数据对齐，通过选择正确的行和列
    heads_pruned = box_df["Heads Pruned"]
    box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 50 x 25 的箱线图数据

    # 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
    line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)

    # 计算用于 x 轴标签的修剪百分比
    percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)

    # Generate data
    heads_pruned = np.arange(0, 150, 6)
    accuracy_no_pruning = 0.968226706741091
    accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
    accuracy_random = box_data

    # Calculate percentage pruned for x-axis labels
    percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

    fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')), row, col)

    # No Pruning line
    fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                            mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')), row, col)

    for i, h in enumerate(heads_pruned):
        fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                            marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')), row, col)

    fig.add_trace(go.Scatter(
        x=[None], y=[None],  
        mode='markers',
        marker=dict(color='red'),
        name='Outliers Pruning'  
    ), row, col)

    fig.update_xaxes(
    title_text="Heads Pruned (%)",
    showgrid=True,
    tickvals=heads_pruned_bert[::3],
    ticktext=[f"{p}%" for h, p in zip(heads_pruned_bert[::3], percent_pruned[::3])],
    tickmode='array',
    row=1, col=1
    )

    fig.update_yaxes(title_text="Accuracy",showgrid=True, range=[0.0, 1.0],row=1,col=1)

def create_model_plots_ALBERT(fig, row, col, box_data_path, line_data_path):
    # 读取箱线图数据 (100x25)
    box_df = pd.read_excel(box_data_path)

    # 读取折线图数据 (144 个点用于修剪 1 到 144 个头)
    line_df = pd.read_csv(line_data_path, header=None)
    line_data = line_df.squeeze()  # 转换为 Series

    # 确保数据对齐，通过选择正确的行和列
    heads_pruned = box_df["Heads Pruned"]
    box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 25 x 13 的箱线图数据

    # 从折线图数据中选择每第 6 个点，以匹配箱线图的 x 轴比例
    line_data_sampled = line_data.iloc[1:].reset_index(drop=True)

    # 计算用于 x 轴标签的修剪百分比
    percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)

    # Generate data
    heads_pruned = np.arange(0, 150, 12)
    accuracy_no_pruning = 0.8448293428425879
    accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
    accuracy_random = box_data

    # Calculate percentage pruned for x-axis labels
    percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

    # A* Search line
    fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')), row, col)

    # No Pruning line
    fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning],
                            mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')), row, col)

    # Random pruning box plot
    for i, h in enumerate(heads_pruned):
        fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning',
                            marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')), row, col)

    fig.add_trace(go.Scatter(
        x=[None], y=[None],  # 使用 None 作为 x 和 y 值，使其不会显示在图中
        mode='markers',
        marker=dict(color='red'),
        name='Outliers Pruning'  # 设置图例标签
    ), row, col)

    fig.update_xaxes(
    title_text="Heads Pruned (%)",
    showgrid=True,
    tickvals=heads_pruned[::2],
    ticktext=[f"{p}%" for h, p in zip(heads_pruned[::2], percent_pruned[::2])],
    tickmode='array',
    row=1, col=2
    )

    fig.update_yaxes(title_text="Accuracy",showgrid=True, range=[0.0, 1.0],row=1,col=2)

# BERT
heads_pruned_bert = np.arange(0, 150, 6)
accuracy_no_pruning_bert = 0.968226706741091
create_model_plots_BERT(fig, 1, 1, box_data_path_BERT, line_data_path_BERT)

# ALBERT
heads_pruned_albert = np.arange(0, 150, 12)
accuracy_no_pruning_albert = 0.8448293428425879
create_model_plots_ALBERT(fig, 1, 2, box_data_path_ALBERT, line_data_path_ALBERT)

# # RoBERTa
# heads_pruned_roberta = np.arange(0, 384, 16)
# accuracy_no_pruning_roberta = 0.878
# accuracy_random_roberta = box_data
# create_model_plots(fig, 2, 1, box_data_path_ROBERTA, line_data_path_ROBERTA, heads_pruned_roberta, accuracy_no_pruning_roberta, accuracy_random_roberta)

# # XLM-RoBERTa
# heads_pruned_xlm = np.arange(0, 150, 6)
# accuracy_no_pruning_xlm = 0.9973
# accuracy_random_xlm = box_data
# create_model_plots(fig, 2, 2, box_data_path_XLM, line_data_path_XLM, heads_pruned_xlm, accuracy_no_pruning_xlm, accuracy_random_xlm)

# 更新布局
fig.update_layout(
    title="Accuracy During Heads Pruning for Different Models",
    showlegend=False,
    template="plotly_white",
    width=900,
    height=800
    )

fig.update_yaxes(title_text="Accuracy",showgrid=True, range=[0.3, 1.0],row=1,col=1)
fig.update_yaxes(title_text="Accuracy",showgrid=True, range=[0.3, 1.0],row=1,col=2)


fig.show()


In [51]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# 读取数据 (以 BERT 数据为例，其他模型数据类似)
# 数据路径
box_data_path_BERT = "experiments_results/BERTrandompruned_heads_data_50x25_concated.xlsx"
line_data_path_BERT = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"
box_data_path_ALBERT = "experiments_results/ALBERTrandompruned_heads_data_25x13_1group.xlsx"
line_data_path_ALBERT = "experiments_results/pruned_heads_accuracy(ALBERT)basedonGreedyGnorm.csv"
box_data_path_ROBERTA = "experiments_results/ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_ROBERTA = "experiments_results/pruned_heads_accuracy(ROBERTA)basedonGreedyGnorm.csv"
box_data_path_XLM = "experiments_results/XLM_ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_XLM = "experiments_results/pruned_heads_accuracy(XLM_ROBERTA)basedonGreedyGnorm.csv"

# 创建 4 个子图
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("BERT", "ALBERT", "RoBERTa", "XLM-RoBERTa"),
    shared_yaxes=True,
    vertical_spacing=0.1
)

def create_model_plots_BERT(fig, row, col, box_data_path, line_data_path):
    box_df = pd.read_excel(box_data_path)
    line_df = pd.read_csv(line_data_path, header=None)
    line_data = line_df.squeeze()  # 转换为 Series
    heads_pruned = box_df["Heads Pruned"]
    box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 50 x 25 的箱线图数据
    line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)
    percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)
    np.random.seed(0)
    heads_pruned = np.arange(0, 150, 6)
    accuracy_no_pruning = 0.968226706741091
    accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
    accuracy_random = box_data

    # Add traces to figure
    fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')), row=row, col=col)
    fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning], mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')), row=row, col=col)

    for i, h in enumerate(heads_pruned):
        fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning', marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')), row=row, col=col)

    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color='red'), name='Outliers Pruning'), row=row, col=col)

def create_model_plots_ALBERT(fig, row, col, box_data_path, line_data_path):
    box_df = pd.read_excel(box_data_path)
    line_df = pd.read_csv(line_data_path, header=None)
    line_data = line_df.squeeze()  # 转换为 Series
    heads_pruned = box_df["Heads Pruned"]
    box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取 25 x 13 的箱线图数据
    line_data_sampled = line_data.iloc[1:].reset_index(drop=True)
    percent_pruned = (heads_pruned / 144 * 100).round(0).astype(int)
    heads_pruned = np.arange(0, 150, 12)
    accuracy_no_pruning = 0.8448293428425879
    accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]
    accuracy_random = box_data

    # Add traces to figure
    fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')), row=row, col=col)
    fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning], mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')), row=row, col=col)

    for i, h in enumerate(heads_pruned):
        fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning', marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')), row=row, col=col)

    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color='red'), name='Outliers Pruning'), row=row, col=col)

# 绘制图
create_model_plots_BERT(fig, 1, 1, box_data_path_BERT, line_data_path_BERT)
create_model_plots_ALBERT(fig, 1, 2, box_data_path_ALBERT, line_data_path_ALBERT)

# RoBERTa and XLM-RoBERTa would be added similarly here...

# 更新布局
fig.update_layout(
    title="Accuracy During Heads Pruning for Different Models",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    showlegend=True,
    legend=dict(x=1, y=1),  # 将图例放在右侧
    template="plotly_white",
    width=1200,
    height=800
)

fig.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

# 创建函数用于绘制单个模型的图表
def create_model_plot(box_data_path, line_data_path, heads_pruned, accuracy_no_pruning, accuracy_random, title):
    box_df = pd.read_excel(box_data_path)
    line_df = pd.read_csv(line_data_path, header=None)
    line_data = line_df.squeeze()  # 转换为 Series

    heads_pruned_data = box_df["Heads Pruned"]
    box_data = box_df.drop(columns=["Heads Pruned"]).values  # 获取箱线图数据

    line_data_sampled = line_data.iloc[1::6].reset_index(drop=True)  # 选择每第 6 个点
    accuracy_greedyGnorm = [float(_) for _ in list(line_data_sampled[1])]

    # 计算修剪百分比
    percent_pruned = ((heads_pruned / 144) * 100).round(0).astype(int)

    # 创建图形
    fig = go.Figure()

    # 绘制 A* 搜索线
    fig.add_trace(go.Scatter(x=heads_pruned, y=accuracy_greedyGnorm, mode='lines', name='Greedy Gnorm', line=dict(color='green')))

    # 绘制无修剪线
    fig.add_trace(go.Scatter(x=[heads_pruned[0], heads_pruned[-1]], y=[accuracy_no_pruning, accuracy_no_pruning], 
                             mode='lines', name='No Pruning', line=dict(color='orange', dash='dash')))

    # 绘制随机修剪箱线图
    for i, h in enumerate(heads_pruned):
        fig.add_trace(go.Box(y=accuracy_random[i], x=[h] * len(accuracy_random[i]), name='Random Pruning', 
                             marker_color='red', showlegend=(i == 0), line=dict(color='skyblue')))

    # 添加图例
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color='red'), name='Outliers Pruning'))

    fig.update_layout(
        title=title,
        xaxis_title="Heads Pruned",
        yaxis_title="Accuracy",
        showlegend=False,
        template="plotly_white",
        width=900,
        height=600,
        yaxis=dict(showgrid=True, range=[0.5, 1.0])  # 调整 y 轴的范围
        
    )

    return fig

# 数据路径
box_data_path_BERT = "experiments_results/BERTrandompruned_heads_data_50x25_concated.xlsx"
line_data_path_BERT = "experiments_results/pruned_heads_accuracy(BERT)basedonGreedyGnorm.csv"
box_data_path_ALBERT = "experiments_results/ALBERTrandompruned_heads_data_25x13_1group.xlsx"
line_data_path_ALBERT = "experiments_results/pruned_heads_accuracy(ALBERT)basedonGreedyGnorm.csv"
box_data_path_ROBERTA = "experiments_results/ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_ROBERTA = "experiments_results/pruned_heads_accuracy(ROBERTA)basedonGreedyGnorm.csv"
box_data_path_XLM = "experiments_results/XLM_ROBERTArandompruned_heads_data_25x25_1group.xlsx"
line_data_path_XLM = "experiments_results/pruned_heads_accuracy(XLM_ROBERTA)basedonGreedyGnorm.csv"

# 设置不同模型的参数
heads_pruned_bert = np.arange(0, 150, 6)
accuracy_no_pruning_bert = 0.968226706741091
accuracy_random_bert = np.random.rand(len(heads_pruned_bert))  # 模拟的随机修剪数据

heads_pruned_albert = np.arange(0, 150, 12)
accuracy_no_pruning_albert = 0.8448293428425879
accuracy_random_albert = np.random.rand(len(heads_pruned_albert))  # 模拟的随机修剪数据

heads_pruned_roberta = np.arange(0, 384, 16)
accuracy_no_pruning_roberta = 0.878
accuracy_random_roberta = np.random.rand(len(heads_pruned_roberta))  # 模拟的随机修剪数据

heads_pruned_xlm = np.arange(0, 150, 6)
accuracy_no_pruning_xlm = 0.9973
accuracy_random_xlm = np.random.rand(len(heads_pruned_xlm))  # 模拟的随机修剪数据

# 分别绘制四个图
fig_bert = create_model_plot(box_data_path_BERT, line_data_path_BERT, heads_pruned_bert, accuracy_no_pruning_bert, accuracy_random_bert, "BERT")
fig_albert = create_model_plot(box_data_path_ALBERT, line_data_path_ALBERT, heads_pruned_albert, accuracy_no_pruning_albert, accuracy_random_albert, "ALBERT")
fig_roberta = create_model_plot(box_data_path_ROBERTA, line_data_path_ROBERTA, heads_pruned_roberta, accuracy_no_pruning_roberta, accuracy_random_roberta, "RoBERTa")
fig_xlm = create_model_plot(box_data_path_XLM, line_data_path_XLM, heads_pruned_xlm, accuracy_no_pruning_xlm, accuracy_random_xlm, "XLM-RoBERTa")

# 合并图形到一个图
fig_combined = make_subplots(
    rows=2, cols=2,
    subplot_titles=("BERT", "ALBERT", "RoBERTa", "XLM-RoBERTa"),
    shared_yaxes=True,
    vertical_spacing=0.1
)

# 将每个模型的图添加到相应的子图
fig_combined.add_trace(fig_bert.data[0], row=1, col=1)
fig_combined.add_trace(fig_bert.data[1], row=1, col=1)
fig_combined.add_trace(fig_bert.data[2], row=1, col=1)

fig_combined.add_trace(fig_albert.data[0], row=1, col=2)
fig_combined.add_trace(fig_albert.data[1], row=1, col=2)
fig_combined.add_trace(fig_albert.data[2], row=1, col=2)

fig_combined.add_trace(fig_roberta.data[0], row=2, col=1)
fig_combined.add_trace(fig_roberta.data[1], row=2, col=1)
fig_combined.add_trace(fig_roberta.data[2], row=2, col=1)

fig_combined.add_trace(fig_xlm.data[0], row=2, col=2)
fig_combined.add_trace(fig_xlm.data[1], row=2, col=2)
fig_combined.add_trace(fig_xlm.data[2], row=2, col=2)

# 更新布局
fig_combined.update_layout(
    title="Accuracy During Heads Pruning for Different Models",
    xaxis_title="Heads Pruned",
    yaxis_title="Accuracy",
    showlegend=False,
    template="plotly_white",
    width=1200,
    height=800
)



fig_combined.show()


TypeError: object of type 'numpy.float64' has no len()